In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [49]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Estimated Salary will be a Output/ Dependant Feature which is continous variable, hence, regression problem

In [50]:
# Drop unnecessary columns
data= data.drop(['RowNumber', 'CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [51]:
# Label Encoder for Gender Column
le_gender= LabelEncoder()
data['Gender']= le_gender.fit_transform(data['Gender'])

In [52]:
# OHE for Geography
ohe_geography= OneHotEncoder(handle_unknown='ignore')
geo_encoded= ohe_geography.fit_transform (data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded, columns=ohe_geography.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [53]:
data= pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [54]:
# Split Data into Dependent and Independant Features

X= data.drop('EstimatedSalary', axis=1)
y= data['EstimatedSalary']

In [55]:
# Train test Split
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=123)

In [ ]:
# Scaling Features

std_scaler= StandardScaler()
X_train= std_scaler.fit_transform(X_train)
X_test= std_scaler.transform(X_test)

In [58]:
# Save Encoders and Scaler
with open('le_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)

with open('ohe_geography', 'wb') as file:
    pickle.dump(ohe_geography,file)

with open('std_scaler.pkl','wb') as file:
    pickle.dump(std_scaler, file)

## ANN Regression Problem statement

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [65]:
# Build Model

model= Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # By Default Linear Activation function will work
])

In [88]:
# Model Compiling

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [89]:
## Set up the Tensorboard

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir= "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)

In [90]:
# Set up early stopping

early_stopping_callback= EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [91]:
history= model.fit(
    X_train,y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tf_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 49505.0352 - mae: 49505.0352 - val_loss: 49539.0781 - val_mae: 49539.0781
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 49497.4688 - mae: 49497.4688 - val_loss: 49543.6055 - val_mae: 49543.6055
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 49497.8438 - mae: 49497.8438 - val_loss: 49541.0508 - val_mae: 49541.0508
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 49495.9883 - mae: 49495.9883 - val_loss: 49545.6562 - val_mae: 49545.6562
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 49492.7578 - mae: 49492.7578 - val_loss: 49549.0781 - val_mae: 49549.0781
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 49491.3125 - mae: 49491.3125 - val_loss: 49535.7734 - val_mae: 49535.7734
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 49489.3398 - mae: 49489.3398 - 

In [92]:
## Load tensorBoard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [93]:
%tensorboard --logdir regressionlogs/fit --port 6006

Reusing TensorBoard on port 6006 (pid 21480), started 0:08:12 ago. (Use '!kill 21480' to kill it.)

In [94]:
# Evaluate the model on Test Data

test_loss,test_mae= model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

 1/63 [..............................] - ETA: 1s - loss: 45944.9492 - mae: 45944.9492

63/63 [==============================] - 0s 1ms/step - loss: 49535.1758 - mae: 49535.1758
Test MAE: 49535.17578125
